## 下载数据集到本地

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("naturebalance/chinesemedicalcollection")

print("Path to dataset files:", path) # 打印数据集文件路径

100%|██████████| 186M/186M [00:24<00:00, 8.03MB/s] 

Extracting model files...


Path to dataset files: /home/jacko/.cache/kagglehub/datasets/naturebalance/chinesemedicalcollection/versions/22


## 将所有数据移动到当前文件夹下，方便后续处理

In [5]:
!mv $path ./

## 读取所有的书籍索引信息

In [6]:
import pandas as pd
import numpy as np

namelist = pd.read_csv('./22/nameList.txt')

namelistnp = np.array(namelist)

print(namelistnp)

[[0 '神农本草经']
 [1 '吴普本草']
 [2 '本草经集注']
 ...
 [697 '止园医话']
 [698 '脉诀阐微 脉学阐微']
 [699 '名老中医之路']]


## 过滤不需要的书籍

In [7]:
filterOut = []

for it in namelistnp:
    if "黄帝" in it[1] or "伤寒" in it[1]:
        filterOut.append(list(it))

print(filterOut)

[[346, '黄帝明堂灸经'], [430, '黄帝内经素问集注'], [431, '黄帝内经灵枢集注'], [435, '黄帝素问直解'], [437, '黄帝内经素问'], [438, '黄帝内经素问校义'], [439, '黄帝内经太素'], [451, '黄帝内经素问遗篇'], [453, '黄帝素问宣明论方'], [457, '伤寒论'], [458, '伤寒捷诀'], [459, '伤寒总病论'], [461, '注解伤寒论'], [462, '伤寒九十论'], [463, '伤寒百证歌'], [464, '伤寒发微论'], [465, '伤寒明理论'], [466, '仲景伤寒补亡论'], [467, '伤寒寻源'], [468, '伤寒直格'], [469, '伤寒标本心法类萃'], [470, '伤寒六书'], [471, '伤寒论条辨'], [472, '张卿子伤寒论'], [473, '伤寒证治准绳'], [474, '伤寒论注'], [475, '重订通俗伤寒论'], [476, '伤寒溯源集'], [477, '伤寒括要'], [478, '伤寒缵论'], [479, '伤寒贯珠集'], [480, '伤寒法祖'], [481, '伤寒大白'], [482, '伤寒悬解'], [483, '伤寒论类方'], [484, '伤寒论辩证广注'], [485, '伤寒论辑义'], [486, '伤寒医诀串解'], [487, '伤寒审证表'], [488, '伤寒补例'], [489, '敖氏伤寒金镜录'], [490, '伤寒舌鉴'], [492, '伤寒论翼'], [493, '伤寒附翼'], [494, '伤寒指掌'], [496, '河间伤寒心要'], [497, '刘河间伤寒医鉴'], [609, '伤寒论纲目'], [633, '证治准绳·伤寒'], [638, '订正仲景全书伤寒论注'], [646, '伤寒心法要诀'], [654, '伤寒辨要笺记'], [690, '黄帝内经十二经脉秘与应用']]


## 构建数据集

In [9]:
import os
import re
import json

basePath = "./22/chinesemedicalbook/ChineseMedicalBook"
pattern = re.compile("<篇名>(.*?)<目录>", re.DOTALL)
jsonPath = "./chinesemedicalbook.json"

jsonList = []

def get_content(text):
    match = pattern.findall(text)
    if len(match) == 0:
        return None
    return match
   
for it in filterOut:
    path = os.path.join(basePath, str(it[0])+".txt")
    with open(path, "r") as f:
        text = f.read()
        content = get_content(text)
        if content is None:
            print("Unsolved content in file:", path)
            continue
        for i in range(1, len(content)):
            content[i] = content[i].strip()
            contentList = content[i].split("\n")
            for j in range(len(contentList)):
                contentList[j] = contentList[j].strip()
            contentMerge = contentList[0] + "\n" + "".join(contentList[1:])
            contentMerge = re.sub(r'\\x(.*?)\\x', r'<\1>', contentMerge)
            contentMerge = re.sub(r'\\r(.*?)\\r', r'', contentMerge)
            jsonList.append({"text" : contentMerge})
            # print(contentMerge)

with open(jsonPath, "w") as f:
    f.write(json.dumps(jsonList, ensure_ascii=False))
    print("Done!")

Unsolved content in file: ./22/chinesemedicalbook/ChineseMedicalBook/654.txt
Unsolved content in file: ./22/chinesemedicalbook/ChineseMedicalBook/690.txt
Done!
